<a href="https://colab.research.google.com/github/winlam345/UTS_ML2019_ID98125216/blob/master/Assign2_ID3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Pratical Machine Learning Project

## Introduction

ID3 short for iterative Dichotomiser 3, is a decision tree learning algorithm created by Ross Quinlan. As a learning algorithm, ID3 uses labelled data to construct decision trees that later can be used for classification of unseen data. 

ID3 is unique in that it is a greedy algorithm. This means that the algorithm constructs a decision tree in a top down manner; starting from the root down to the leaf nodes where each successive addition is chosen as best option at the time. As such, the decision tree produced is unlikely to be most optimal classifer.

ID3 uses information gain as a metric to determine the best condition to separate the dataset. Information gain is the reduction of overall entropy.

### Input
This implementation of the ID3 algorithm requires the user to provide the column name of the class label and a dataset. The dataset is split into a training set and test set respectively.



In [0]:
import seaborn as sns
# userinput: change sns.load_dataset('iris') to desired dataset
dataset = sns.load_dataset('iris')
train_set, test_set = create_train_test(dataset, 0.8)

# user input: change 'species' to dsired class label
target_attribute = 'species'

decision_tree = train_tree(train_set, target_attribute) # need to run from methodology onward first

### Output
This implemenation outputs a tree structure that can be used to find accuracy value and provide modified  appends prediction to test data set.

In [80]:
print(decision_tree)
print(accuracy(test_set, tree, target_attribute))
print(test_set.head())

{'petal_width <= 0.5': ['setosa', {'petal_length <= 6.4': ['versicolor', 'virginica']}]}
0.7666666666666667
    sepal_length  sepal_width  petal_length  ...  species prediction comparison
9            4.9          3.1           1.5  ...   setosa     setosa       True
14           5.8          4.0           1.2  ...   setosa     setosa       True
19           5.1          3.8           1.5  ...   setosa     setosa       True
21           5.1          3.7           1.5  ...   setosa     setosa       True
25           5.0          3.0           1.6  ...   setosa     setosa       True

[5 rows x 7 columns]


## Exploration

### Dataset

In [89]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


The iris flower dataset is a well known dataset that is widely used for educational and demonstrative purposes for teaching machine learning. As such all 150 instances have no null values and the entire dataset requires little preprocessing. This is useful for focusing on a implementation of the ID3 algorithm as there is no need to remove or estimate missing values.

The dataset itself contains various measurements of various iris flowers organised into four attributes (sepal_length, sepal_width, petal_width, petal_length, species) and one class label (species). The class labels are not continous values meaning a classification tree is needed rather than a regression tree. This will mean that this dataset can test make use of implemented ID3 aglorithm. But the attributes values are continous values and requires modification in the implementation to handle these cases.

## Challenges
The intial challenges encountered in the implementation of ID3 algorithm was how to represent a tree structure in python. There are many methods to represent a tree structure in python. As a novice programmer building a node class and node functionality was quite difficult to grasp. Being not able to handle further errors or bugs required me to find a simplier solution to represent trees for ID3 algorithm. A choice was to restrict the underlying tree structure of the decision tree to a binary tree. A binary tree is a type of tree structure that limits the amount of child nodes a parent node can have to two. This would also help with handling continous data attributes. 

### Data Structure

The datastructure binary tree of implementation would constructed using dictionaries. A dictionary would respresent a sub-tree where the key would be the parent node and the value would be a two element list to represent two possible children nodes. To represent depth in a binary tree, one of the child nodes may also be a sub-tree.

In [0]:
dictionary = {'key': ['element_1', 'element_2']}
#binary_tree = {parent_node: [child_1, child_2]}
#depth2 tree
#binary_tree = {parent_node_1: [child_1, {parent_node_2: [child_2, child_3]} ]}

## Methdology 

### ID3 psuedo code provided

In [101]:
"""
Pseudocode ID3 (Examples, Target_Attribute, Attributes)

  Create a root node for the tree

  If all examples are positive, Return the single-node tree Root, with label = +.
  If all examples are negative, Return the single-node tree Root, with label = -.
  If number of predicting attributes is empty, then Return the single node tree Root, with label = most common value of the target attribute in the examples.

  Otherwise Begin

      
      A ← The Attribute that best classifies examples.
      Decision Tree attribute for Root = A.

      For each possible value, vi, of A,
          Add a new tree branch below Root, corresponding to the test A = vi.

          Let Examples(vi) be the subset of examples that have the value vi for A

          If Examples(vi) is empty

              Then below this new branch add a leaf node with label = most common target value in the examples

          Else below this new branch add the subtree ID3 (Examples(vi), Target_Attribute, Attributes – {A})

  End
  Return Root
"""

'\nPseudocode ID3 (Examples, Target_Attribute, Attributes)\n\n  Create a root node for the tree\n\n  If all examples are positive, Return the single-node tree Root, with label = +.\n\n  If all examples are negative, Return the single-node tree Root, with label = -.\n\n  If number of predicting attributes is empty, then Return the single node tree Root, with label = most common value of the target attribute in the examples.\n\n  Otherwise Begin\n\n      A ← The Attribute that best classifies examples.\n      Decision Tree attribute for Root = A.\n\n      For each possible value, vi, of A,\n          Add a new tree branch below Root, corresponding to the test A = vi.\n\n          Let Examples(vi) be the subset of examples that have the value vi for A\n\n          If Examples(vi) is empty\n\n              Then below this new branch add a leaf node with label = most common target value in the examples\n\n          Else below this new branch add the subtree ID3 (Examples(vi), Target_Attribu

Pseudo Code was understood as in the following images:

https://drive.google.com/file/d/1_R56zO81NUfBt83mJyKf4g_gpMnS7rIm/view?usp=sharing

# Implementation

### Import Statements

--- Start run here(Ctrl+F10)---

In [0]:
import seaborn as sns
import numpy as np
import pandas as pd


dataset = sns.load_dataset('iris')

### Create Train and Test Datasets

To perform classification with a decision tree using ID3, the dataset must be first be split into training and test data

In [0]:
def create_train_test(dataset, split_ratio):
  """
  Generates two subets from the main dataset according to specified a ratio
  """
  dataset_copy = dataset
  training_set = dataset_copy.sample(frac = split_ratio, random_state=0)
  testing_set = dataset_copy.drop(training_set.index)
    
  return training_set, testing_set

train_set, test_set = create_train_test(dataset, 0.8)

## ID3 Algorithm

In [0]:
def id3(dataset, attributes, target_attribute):
  """
  There are param for id3 algorithm.
  dataset: dataframe of current subset
  attributes: list of attributes found in current subset
  target_attribute: string of class label
  """
  
  # condition_1 to stop recursion
  if check_homogenous(dataset, target_attribute):
    class_labels = dataset[target_attribute].unique()
    return class_labels[0]
  
  else:
      
      # identify all possible values to seperate dataset on
      potential_splits = find_potential_splits(attributes, dataset)
      
      # find best attribute and attribute value out of possible splits
      best_attribute, best_attribute_value = find_best_split(dataset, attributes, target_attribute, potential_splits)
      
      #seperate the data accordingly
      less_equal_than, greater_than = split_data(dataset, best_attribute, best_attribute_value)
        
      # condition_2 to stop recursion
      if check_empty(less_equal_than, greater_than):
        leaf_node = make_leaf_node(dataset, target_attribute)
        return leaf_node
      
      #construct tree 
      split_condition = "{} <= {}".format(best_attribute, best_attribute_value)
      binary_tree = {split_condition: []}
      
      # recursion
      satisfy_subset = id3(less_equal_than, attributes, target_attribute)
      failed_subset = id3(greater_than, attributes, target_attribute)
      binary_tree[split_condition].append(satisfy_subset)
      binary_tree[split_condition].append(failed_subset)
      
      return binary_tree

### Condition_1

"if every element in the subset belongs to the same class; in which case the node is turned into a leaf node and labelled with the class of the examples"

In [0]:
def check_homogenous(dataset, target_attribute):
    """
    Asks if the current dataset only contains one class label value
    """
    class_labels = dataset[target_attribute].unique()
    if len(class_labels) == 1:
        return True
    else:
        return False

### Condition_2

"there are no more attributes to be selected, but the examples still do not belong to the same class. In this case, the node is made a leaf node and labelled with the most common class of the examples in the subset."

In [0]:
def check_empty(subset, subset_2):
  """
  asks if there are any more attributes in the subsets to be selected
  """
  if (len(subset) == 0 or len(subset_2) == 0):
    return True
  else:
    return False

In [0]:
def make_leaf_node(dataset, target_attribute):
  """
  returns the most common class label in the current dataset and turn into leaf node
  """
  class_labels, class_labels_count = np.unique(dataset[target_attribute], return_counts=True)
    
  index = class_labels_count.argmax()
  leaf_node = class_labels[index]
    
  return leaf_node

### find_potential_splits(attribute, dataset)

In [0]:
def find_potential_splits(attribute_list, dataset):
  """
  returns a dictionary of all unique possible values
  key: attribute
  value: list of corresponding attribute values
  """
  potential_splits = {}
  
  for i in range(len(attribute_list)):
    potential_splits[attribute_list[i]] = []
    unique_values = train_set[attribute_list[i]].unique()
    potential_splits[attribute_list[i]].append(unique_values)
        
  return potential_splits

### Split_Data


In [0]:
def split_data(dataset, best_attribute, best_value):
  """
  takes current dataset and binary split into two subset
  """
  values = dataset[best_attribute]
  less_equal_than = dataset[values <= best_value]
  greater_than = dataset[values > best_value]
    
  return less_equal_than, greater_than

### Find_bestsplit

Find best attribute and best attribute value using information gain as the metric to compare

### Entropy

In [0]:
def entropy(dataset, target_attribute):
  class_labels = dataset[target_attribute]
  _, counts = np.unique(class_labels, return_counts = True)
    
  probabilities = counts / counts.sum()
    
  entropy = sum(probabilities * -np.log2(probabilities))
    
  return(entropy)

### Information Gain

In [0]:
def information_gain(less_equal_than, greater_than, target_attribute):
    total_data = len(less_equal_than) + len(greater_than)
    prob_less_equal_than = len(less_equal_than) / total_data
    prob_greater_than = len(greater_than) / total_data
    
    information_gain = prob_less_equal_than * entropy(less_equal_than, target_attribute) + (prob_greater_than * entropy(greater_than, target_attribute))
    
    return information_gain
    

In [0]:
def find_best_split(dataset, attribute_list, target_attribute, potential_splits):
  """
  # find best attribute and attribute value out of possible splits
  # best is highest information gain
  """
  best_info_gain = 99999 # arbituary large number
    
  for attribute in potential_splits:
      for dict_value in potential_splits[attribute]:
          #taking array object out of dictionary value
          attribute_values = dict_value
          for value in attribute_values:
              less_equal_than, greater_than = split_data(dataset, attribute, value)
              current_gain = information_gain(less_equal_than, greater_than, target_attribute)
                
          if current_gain <= best_info_gain :
              best_info_gain = current_gain
              best_attribute = attribute
              best_attribute_value = value
    
    
  return best_attribute, best_attribute_value

# Train a decision tree using ID3
Main function to call to create decision tree by user

In [0]:
def train_tree(train_set, target_attribute):
  """
  takes user input and prepares parameters
  """
  attribute = list(train_set.columns)
  attribute.remove(target_attribute)
  
  decision_tree = id3(train_set, attribute, target_attribute)

  return decision_tree

#tree = train_tree(train_set, 'species')
#print(tree)

# Evaluation

In [0]:
def classify(instance, tree):
    """
    compare prediction of tree with test data
    """
    split_condition = list(tree.keys())[0]
    attribute, operand, value = split_condition.split()
    
    if instance[attribute] <= float(value):
        label = tree[split_condition][0]
    else:
        label = tree[split_condition][1]
    
    if not isinstance(label, dict):
        return label
    else:
        remaining_tree = label
        return classify(instance, remaining_tree)

In [0]:
def accuracy(unknown_dataset, tree, target_attribute):
    #create column of tree's prediction
    unknown_dataset['prediction'] = unknown_dataset.apply(classify,  args=(tree,), axis=1)
    unknown_dataset['comparison'] = unknown_dataset.prediction == unknown_dataset[target_attribute]
                    
    accuracy = unknown_dataset.comparison.mean()
                    
    return accuracy

## Accuracy

In [85]:
decision_tree = train_tree(train_set, target_attribute)
accuracy(test_set, tree, target_attribute)

0.7666666666666667

## Time

In [86]:
time = %time decision_tree = train_tree(train_set, target_attribute)

CPU times: user 583 ms, sys: 2.46 ms, total: 586 ms
Wall time: 595 ms


# Conclusion

Overall the accuracy of my implementation of ID3 is rather low with accuracy of 77%. The run time is fast but the dataset is rather small only containg few instances. 

To futher improve this implementation extra features of maximum depth and could be implemented. Maximum depth would limit how deep a tree may be and be used as tuning parameter to prevent over fitting the model. 

Another improvement to implementation is to properly implement a tree data structure rather than just a binary tree. This would allow for possibility for many splits to occur per split condition.

Other improvements would be to include a data split function that includes a cross validation dataset and fit function to optimise any future implemented parameters of this algorithm. This would allow for construction of better models on complicated data and reduce the chance of over-fitting.

# Ethics

Ethics is defined as the moral principles that govern a person's behaviour or the conducting of an activity. As such to look we must look at possible applications of a decision tree and the implications of their use. Decision trees can effectively model possible decisions and outcomes and such are used in decision making applications. These applications can in theory vary from very pragmatic uses such as churn analysis and marketing selection to very serious that involve patient medical data. Decision trees may be used to diagnose patient or administer medications according to symptoms. In its nature, decision trees and machine learning operate probablistically and will contain errors. In the event that an error occurs that puts a person's life at risk provided by a decision tree, is this acceptable? 

From a utilitarian approach, the good of the many is superior the the good of the few. In theory, a commercial decision algorithm applicationn will have rather low error rate. Although few people are put at risk, a larger population of people will benefit from use of decision trees and as such is it acceptable to use decision trees to handle life or death situations from a ultilitarian view.